In [ ]:
from utils import DatabaseManager, UserAgentManager, Scraper

from entities import Match
import logging

from typing import List
from tqdm.auto import tqdm

import asyncio

import nest_asyncio
nest_asyncio.apply()

import logging
logging.getLogger('stem').setLevel(logging.CRITICAL)


In [ ]:
db_manager = DatabaseManager('../data/volleyball_world.sqlite')
user_agent_manager = UserAgentManager(
    password='password', logger_level=logging.INFO)

match_ids = list(range(1, 10000+1))

In [ ]:
async with Scraper(domain='https://en.volleyballworld.com/',
                       uam=user_agent_manager,
                       min_delay=2,
                       max_delay=5,
                       logger_level=logging.ERROR) as scraper:
        for match_id in tqdm(match_ids):
            page_path = f'volleyball/competitions/volleyball-nations-league/schedule/{match_id}'
            while True:
                try:
                    page = (await scraper.get_page(page_path))[0]
                    break
                except Exception as e:
                    print (f'Exception occured fetching match #{match_id} -- {e}. Retrying.')
            match_obj = Match(match_id, page)
            match_info = match_obj.get_match_info()
            teams_in_match_info = match_obj.get_team_in_match_info()
            teams_in_set_info = match_obj.get_team_in_set_info()
            players_statistics = match_obj.get_players_results_info()

            db_manager.insert(match_info=match_info,
                              teams_in_match_info=teams_in_match_info,
                              teams_in_set_info=teams_in_set_info,
                              players_statistics=players_statistics)       